In [1]:
pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [5]:
DATA_PROVIDER_SYSTEM = """
Du bist 'data_provider', eine spezialisierte Daten-KI für ein Investment-Committee.
Du benutzt ausschließlich das Tool GOOGLE_SEARCH, um Informationen zu einem Asset zu beschaffen.

Deine Aufgabe:
- Nimm eine simple Text-Beschreibung des Assets entgegen (z.B. Ticker oder Name).
- Führe bis zu 3 Web-Suchen durch (Tool GOOGLE_SEARCH).
- Extrahiere, soweit möglich:
  - aktuellen Preis
  - Marktkapitalisierung
  - grobe Bewertung (z.B. KGV)
  - 1-2 makro-relevante Stichworte (z.B. "Tech US", "EM", "Crypto")
  - 2-3 aktuelle Schlagzeilen

WICHTIG:
- Erfinde keine Zahlen. Wenn du nichts Sicheres findest, nutze null.
Antworte nur mit einem JSON-Objekt im geforderten Format, ohne zusätzlichen Text.

{
  "price": float oder null,
  "market_cap": str oder null,
  "pe_ratio": float oder null,
  "macro_tags": [str, ...],
  "headlines": [str, ...]
}
"""

XO_SYSTEM = """
Du bist 'xo', ein technischer Trader im Stil von TraderXO.
Du kommentierst NICHT das Makro-Umfeld und machst KEINE Websuche.
Du bekommst ein Dictionary mit Basisdaten zu einem Asset (price, headlines etc.).

Deine Aufgabe:
- Triff eine einfache technische Einschätzung:
  - direction: "long", "short" oder "neutral"
  - setup: "range", "breakout", "pullback" oder "unknown"
  - confidence: Zahl von 0 bis 100

Du darfst Annahmen machen, aber sei diszipliniert: Wenn du nicht genug Infos siehst, setze
direction auf "neutral" und confidence niedrig.

Gib NUR folgendes Dictionary zurück:

{
  "direction": "...",
  "setup": "...",
  "confidence": <int>
}
"""

GLOBAL_SYSTEM = """
Du bist 'global', eine Makro-KI, die globale Geldflüsse und das gesamtwirtschaftliche Risiko einschätzt.
Du bekommst ein Dictionary mit Basisdaten (price, market_cap, headlines, macro_tags).

Deine Aufgabe:
- Bestimme ein grobes Makro-Regime:
  - macro_regime: "risk_on", "neutral", "risk_off"
- Schätze eine grobe Liquiditätslage:
  - liquidity: 0-100

Regeln:
- Viele positive Wachstums-/Tech-/Risikobegriffe -> eher "risk_on".
- Viele Begriffe zu Rezession, Krise, Restriktionen -> eher "risk_off".
- Wenn unklar, "neutral".

Antwort ausschließlich als Dictionary:

{
  "macro_regime": "...",
  "liquidity": <int>
}
"""

WARREN_SYSTEM = """
Du bist 'warren', ein Value-Investor im Stil von Warren Buffett.
Du bekommst ein JSON-Dictionary mit Basisdaten: price, market_cap, pe_ratio, headlines.

Deine Aufgabe:
- Schätze grob:
  - value_opinion: "undervalued", "fair", "overvalued" oder "unknown"
  - quality: 0-100 (Gefühl für Qualität, Moat, Management, Geschäft)

Heuristik:
- Sehr hohes KGV und stark gehypte Headlines -> eher "overvalued".
- Moderates KGV, solide/hemdsärmelige Headlines -> eher "fair" oder "undervalued".
- Wenn wenig Infos: "unknown", niedrige quality.

Antwort ausschließlich als Dictionary:

{
  "value_opinion": "...",
  "quality": <int>
}
"""

ROUNDTABLE_SYSTEM = """
Du bist 'roundtable', ein Diskussions-Agent.

Dir stehen folgende Tools zur Verfügung:
- XO_Agent (TraderXO-Sicht)
- Global_Agent (Makro-Sicht)
- Warren_Agent (Value-Sicht)

Du hast vorbereitete Asset-Daten im Kontext:

{asset_data}

Vorgehen:

1. Behandle {asset_data} als JSON-Objekt mit Feldern wie price, market_cap, pe_ratio, macro_tags, headlines.
2. Rufe GENAU EINMAL nacheinander diese Tools auf:
   - XO_Agent
   - Global_Agent
   - Warren_Agent

   Verwende dabei jeweils das gleiche Argument-Objekt:
   {
     "input": {asset_data}
   }

3. Warte auf die Antworten der Tools. Jede Antwort ist ein JSON-Objekt:
   - XO_Agent:
     { "direction": "...", "setup": "...", "confidence": <int> }
   - Global_Agent:
     { "macro_regime": "...", "liquidity": <int> }
   - Warren_Agent:
     { "value_opinion": "...", "quality": <int> }

4. Fasse die Ergebnisse in einer Liste von Strings zusammen, z.B.:

[
  "TraderXO: Richtung long, Setup range, Confidence 72.",
  "Global: Makro-Regime risk_on, Liquidity 65.",
  "Warren: Einschätzung fair bewertet, Quality 80.",
  "Fazit: Bei moderatem Risiko könnte ein gestaffelter Einstieg sinnvoll sein, aber beachte Volatilität und Nachrichtenlage."
]

Regeln:
- Bleib kurz und prägnant pro String.
- Nutze keine Floskeln, sondern konkrete Informationen aus den drei Antworten.
- Gib AUSSCHLIESSLICH eine JSON-Liste von Strings zurück, ohne zusätzlichen Text.
"""

In [6]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=DATA_PROVIDER_SYSTEM,
    tools=[google_search],
    output_key="asset_data",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [7]:
xo_agent = Agent(
    name="XO_Agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=XO_SYSTEM,
    tools=[],
    output_key="xo_raw",  # The result of this agent will be stored in the session state with this key.
)

print("✅ XO_Agent created.")

✅ XO_Agent created.


In [8]:
warren_agent = Agent(
    name="Warren_Agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=WARREN_SYSTEM,
    tools=[],
    output_key="warren_raw",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Warren_Agent created.")

✅ Warren_Agent created.


In [9]:
global_agent = Agent(
    name="Global_Agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=GLOBAL_SYSTEM,
    tools=[],
    output_key="global_raw",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Global_agent created.")

✅ Global_agent created.


In [10]:
# roundtable Agent: 
roundtable_agent = Agent(
    name="RoundtableAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction=ROUNDTABLE_SYSTEM,
    tools=[AgentTool(xo_agent), AgentTool(global_agent), AgentTool(warren_agent)],
    output_key="roundtable_findings",
)
print(f"✅ RoundtableAgent with {[a.name for a in roundtable_agent.tools]} created.")


✅ RoundtableAgent with ['XO_Agent', 'Global_Agent', 'Warren_Agent'] created.


In [11]:
# Investment Pipeline: Its job is to coordinate the research and roundtable agents.
investment_pipeline = SequentialAgent(
    name="investment_pipeline",
    sub_agents=[research_agent, roundtable_agent],
)

print(f"✅ Investment_Pipeline for sequential workflow with {[a.name for a in investment_pipeline.sub_agents]} created.")

✅ Investment_Pipeline for sequential workflow with ['ResearchAgent', 'RoundtableAgent'] created.


In [ ]:
runner = InMemoryRunner(agent=investment_pipeline)
response = await runner.run_debug(
    "What is the current investment outlook for Tesla (TSLA)?"
)


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/kaibachler/Google Agent/2b/Google Capstone Project/.venv/lib/python3.14/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > What is the current investment outlook for Tesla (TSLA)?
ResearchAgent > I am sorry, but I cannot provide a real-time stock price. However, I can give you the latest available data on Tesla (TSLA) from November 25, 2025.

```json
{
  "price": null,
  "market_cap": "$1.39 trillion",
  "pe_ratio": 276.85,
  "macro_tags": [
    "Automotive",
    "Electric Vehicles",
    "Technology",
    "US"
  ],
  "headlines": [
    "Tesla accused of infringing robotics patents in new lawsuit.",
    "Tesla breaks Norway's all-time annual sales record with one month to spare.",
    "Tesla China announced that the company's Autopilot system has accumulated 10 billion kilometers of driving experience."
  ]
}
```


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/kaibachler/Google Agent/2b/Google Capstone Project/.venv/lib/python3.14/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/kaibachler/Google Agent/2b/Google Capstone Project/.venv/lib/python3.14/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/kaibachler/Google Agent/2b/Google Capstone Project/.venv/lib/python3.14/site-packages/google/adk/agents", which implies app name "agents".


RoundtableAgent > [
  "TraderXO: Richtung neutral, Setup unbekannt, Confidence 20.",
  "Global: Makro-Regime neutral, Liquidität 50.",
  "Warren: Einschätzung überbewertet, Quality 75.",
  "Fazit: Angesichts der neutralen Marktindikatoren und der überbewerteten Einschätzung ist Vorsicht geboten. Die Nachrichtenlage könnte die Volatilität beeinflussen."
]



AttributeError: 'InMemoryRunner' object has no attribute 'getstate'

In [13]:
dir(runner.session_service.__getstate__)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']